#Read the data, drop unneeded columns.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
#mount driver , and unzip , read the pickeled data
from google.colab import drive
drive.mount('/content/gdrive')


In [ ]:
# !unzip '/content/gdrive/MyDrive/3owais/Earning_call/archive.zip'


In [ ]:
unpickled_df = pd.read_pickle("/content/gdrive/MyDrive/3owais_Capital/Earning_call/motley-fool-data.pkl")

pandas_df = pd.DataFrame(unpickled_df)
pandas_df.head(20)

,date,exchange,q,ticker,transcript
0,"Aug 27, 2020, 9:00 p.m. ET",NASDAQ: BILI,2020-Q2,BILI,"Prepared Remarks:\nOperator\nGood day, and wel..."
1,"Jul 30, 2020, 4:30 p.m. ET",NYSE: GFF,2020-Q3,GFF,Prepared Remarks:\nOperator\nThank you for sta...
2,"Oct 23, 2019, 5:00 p.m. ET",NASDAQ: LRCX,2020-Q1,LRCX,Prepared Remarks:\nOperator\nGood day and welc...
3,"Nov 6, 2019, 12:00 p.m. ET",NASDAQ: BBSI,2019-Q3,BBSI,"Prepared Remarks:\nOperator\nGood day, everyon..."
4,"Aug 7, 2019, 8:30 a.m. ET",NASDAQ: CSTE,2019-Q2,CSTE,Prepared Remarks:\nOperator\nGreetings and wel...
5,"Nov 4, 2020, 5:00 p.m. ET",NYSE: GDOT,2020-Q3,GDOT,"Prepared Remarks:\nOperator\nGood afternoon, a..."
6,"Aug 5, 2020, 8:30 a.m. ET",NYSE: KAR,2020-Q2,KAR,Prepared Remarks:\nOperator\nLadies and gentle...
7,"Mar 24, 2021, 11:00 a.m. ET",NYSE: EPAC,2021-Q2,EPAC,Prepared Remarks:\nOperator\nLadies and gentle...
8,"Jun 9, 2021, 8:30 a.m. ET",NYSE: ABM,2021-Q2,ABM,Prepared Remarks:\nOperator\nGreetings and wel...
9,"Aug 11, 2022, 8:30 a.m. ET",NYSE: SGHC,2022-Q2,SGHC,Prepared Remarks:\nOperator\nGood morning and ...


In [ ]:
#drop unnecessary cloumns
train_df = pandas_df.drop(['q','date','exchange','ticker'], axis=1)
train_df.isnull().sum() #no null data

transcript    0
dtype: int64

In [ ]:
#remove some unwanted text at the beginning

text_to_be_removed = "Prepared Remarks:\nOperator\n"
train_df['transcript'] = train_df['transcript'].str.replace(text_to_be_removed, '', regex=False)
train_df['transcript'] = train_df['transcript'].str.replace('\n', '', regex=False)
train_df.head(1000)


,transcript
0,"Good day, and welcome to the Bilibili 2020 Sec..."
1,Thank you for standing by. This is the confere...
2,Good day and welcome to the September 2019 Qua...
3,"Good day, everyone and thank you for participa..."
4,Greetings and welcome to the Caesarstone Limit...
...,...
995,"Good day, ladies and gentlemen, and welcome to..."
996,Welcome to the Wynn Resorts' fourth-quarter 20...
997,"Thank you for standing by, and welcome to the ..."
998,Good morning. My name is Jodie and I will be y...


In [ ]:
train_df['transcript'][2]

'good day and welcome to the september  quarter financial call for lam research at this time id like to turn the conference over to ms tina correia corporate vice president of investor relations please go ahead maamtina correia  vice president investor relations  corporate communicationsthank you operator thank you and good afternoon everyone welcome to the lam research quarterly earnings conference call with me today are tim archer president and chief executive officer and doug bettinger executive vice president and chief financial officerduring todays call we will share our overview on the business environment and review our financial results for the september  quarter and our outlook for the december  quarter the press release detailing our financial results was distributed a little after  oclock pm pacific time this afternoon the release can also be found on the investor relations section of the companys website along with the presentation slides that accompany todays calltodays pr

In [ ]:
lens_of_the_transcripts = [len(text) for text in train_df['transcript_processed']]
average_length = sum(lens_of_the_transcripts) / len(lens_of_the_transcripts)

print(f"Average Transcript Length of each transcript: {average_length}\n")
#!!!!!!!! that's huge !!

#The transcripts are huge , so i will Summarize it about 25% it, get the summarization , apply sentiment analysis on it , based on the sentiment . decide the label , we need to label the data so it can be trained.

In [ ]:
!pip install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.6 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
# from transformers import BertTokenizer, BertForSequenceClassification ,BertLMHeadModel
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
nltk.download("punkt")
nltk.download("stopwords")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
summarizer = pipeline("summarization")
# train_df['summary'] = ""


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
summary_csv =pd.read_csv('/content/gdrive/MyDrive/3owais_Capital/Earning_call/summarized_data.csv')
summarized_data = pd.DataFrame(summary_csv)
#take a copy , don't ruin the original df ://
sampled = summarized_data.copy()
sampled.head(1701)

In [ ]:
# Define a function to preprocess and clean text from stopwords
def clean_text(text):
    stop_words = set(stopwords.words("english"))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    cleaned_text = " ".join(filtered_words)
    return cleaned_text

# Iterate through each row and generate summaries
for index, row in tqdm(sampled[1600:1900].iterrows(), total=len(sampled[0:100])):
    text = row["transcript"][:10000]
    cleaned_text = clean_text(text)  # Clean the text from stopwords

    max_seq_length = 512
    #segment the text to segments , save the segments in segment_summaries list
    segments = [cleaned_text[i:i+max_seq_length] for i in range(0, len(cleaned_text), max_seq_length)]
    segment_summaries = []

    for segment in segments:
        summary = summarizer(segment, max_length=120, min_length=50, do_sample=True)
        segment_summaries.append(summary[0]['summary_text'])

    final_summary = " ".join(segment_summaries) #join all the summaries
    sampled.at[index, "summary"] = final_summary




100%|██████████| 100/100 [1:49:14<00:00, 65.54s/it]


In [ ]:
# sampled.head(20)
# df= pd.DataFrame(sampled)
sampled.head(1801)

,transcript,summary
0,"Good day, and welcome to the Bilibili 2020 Sec...",Bilibili 2020 Second Quarter Earnings Confere...
1,Thank you for standing by. This is the confere...,Griffon Corporation 's Third Quarter 2020 Ear...
2,Good day and welcome to the September 2019 Qua...,"Tim Archer, President Chief Executive Officer..."
3,"Good day, everyone and thank you for participa...",BBSI 's financial results Third Quarter Ended...
4,Greetings and welcome to the Caesarstone Limit...,Caesarstone Limited Second Quarter 2019 Earni...
...,...,...
1796,Good morning and welcome to Otter Tail Corpora...,Otter Tail Corporation 's 2020 Earnings Confe...
1797,"Ladies and gentlemen, thank you for standing b...",Crocs Inc. first-quarter 2020 earnings call ....
1798,Welcome to the FTI Consulting fourth quarter a...,FTI Consulting fourth quarter full year 2021 ...
1799,"Ladies and gentlemen, thank you for standing b...","Matt Cox, Chairman Chief Executive Officer; J..."


In [ ]:
sampled.to_csv("/content/gdrive/MyDrive/3owais_Capital/Earning_call/summarized_data.csv",mode='w' ,index=False)
new_df = pd.read_csv("/content/gdrive/MyDrive/3owais_Capital/Earning_call/summarized_data.csv")
new_df=pd.DataFrame(new_df)

new_df['transcript_processed'] = new_df['transcript'].apply(preprocess_text) #encode stock price
new_df['summary'][1799]
new_df

" Matt Cox, Chairman Chief Executive Officer; Joel Wine, Executive Vice President Chief Financial Officer . Matt Cox and Joel Wine will discuss the Fourth Quarter 2020 Financial Results Conference Call . Slides presentation available download website, www.matson.com Investors tab .  Matson capped strong year continued solid performance Ocean Transportation Logistics despite ongoing challenges pandemic related economic effects . Year-over-year increase operating income Ocean Transportation fourth quarter primarily driven continued exceptional demand CLX CLX+ services . core tradelanes continued see elevated demand sustenance home improvement goods leading higher volume growth Hawaii , Alaska Guam .  Ocean Transportation, contribution CLX CLX+ services primary driver increase operating income year-over-year . Hawaii Guam container volumes approached levels achieved year ago period despite economic challenges pandemic Alaska container volume modestly higher level achieved full year 2019 .

## Analyze the sentiment for summary

In [ ]:
!pip install transformers
!pip install sentencepiece

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the pre-trained BERT model and tokenizer
sa_model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
sa_tokenizer = BertTokenizer.from_pretrained(sa_model_name)
sa_model = BertForSequenceClassification.from_pretrained(sa_model_name)


In [ ]:
summarized = pd.read_csv("/content/gdrive/MyDrive/3owais_Capital/Earning_call/summarized_data.csv")
summarized_data = pd.DataFrame(summarized)
df_forSA = summarized_data[0:1800].copy()
df_forSA.head(1802)

,transcript,summary
0,"Good day, and welcome to the Bilibili 2020 Sec...",Bilibili 2020 Second Quarter Earnings Confere...
1,Thank you for standing by. This is the confere...,Griffon Corporation 's Third Quarter 2020 Ear...
2,Good day and welcome to the September 2019 Qua...,"Tim Archer, President Chief Executive Officer..."
3,"Good day, everyone and thank you for participa...",BBSI 's financial results Third Quarter Ended...
4,Greetings and welcome to the Caesarstone Limit...,Caesarstone Limited Second Quarter 2019 Earni...
...,...,...
1795,"Good afternoon. Thank you for standing by, and...",Wolfspeed Inc. second-quarter fiscal year 202...
1796,Good morning and welcome to Otter Tail Corpora...,Otter Tail Corporation 's 2020 Earnings Confe...
1797,"Ladies and gentlemen, thank you for standing b...",Crocs Inc. first-quarter 2020 earnings call ....
1798,Welcome to the FTI Consulting fourth quarter a...,FTI Consulting fourth quarter full year 2021 ...


In [ ]:
def analyze_sentiment(summary):
    inputs = sa_tokenizer(summary, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = sa_model(**inputs)
    prediction = torch.argmax(outputs.logits).item()
    return prediction #returns a sentioment score from 0 to 5 , 0 and 1 are negative , 3 and up will be positive

# Apply the function to the DataFrame
df_forSA['sentiment'] = df_forSA['summary'].apply(lambda x: analyze_sentiment(x))
#change the sentiment from an integer value to a str value
df_forSA['sentiment'] = df_forSA['sentiment'].map({0: 'negative', 1: 'negative', 2: 'positive' ,3: 'positive' ,4: 'positive',5: 'positive'})
#add stock price based on positive or ngrative sentiment
df_forSA['stock_price'] = df_forSA['sentiment'].apply(lambda x: 'increase' if x == 'positive' else 'decrease')



df_forSA.head(30)

,transcript,summary,sentiment,stock_price
0,"Good day, and welcome to the Bilibili 2020 Sec...",Bilibili 2020 Second Quarter Earnings Confere...,positive,increase
1,Thank you for standing by. This is the confere...,Griffon Corporation 's Third Quarter 2020 Ear...,negative,decrease
2,Good day and welcome to the September 2019 Qua...,"Tim Archer, President Chief Executive Officer...",negative,decrease
3,"Good day, everyone and thank you for participa...",BBSI 's financial results Third Quarter Ended...,negative,decrease
4,Greetings and welcome to the Caesarstone Limit...,Caesarstone Limited Second Quarter 2019 Earni...,negative,decrease
5,"Good afternoon, and welcome to the Green Dot T...",Comments may include forward-looking statemen...,negative,decrease
6,"Ladies and gentlemen, thank you for standing b...","KAR Auction Services, Inc Q2 2020 Earnings Co...",negative,decrease
7,"Ladies and gentlemen, thank you for standing b...",Enerpac Tool Group 's Second Quarter Earnings...,negative,decrease
8,Greetings and welcome to the ABM Industries' S...,ABM Industries ' Second Quarter 2021 Earnings...,positive,increase
9,Good morning and welcome to Super Group's seco...,Super Group 's second quarter 2022 earnings c...,negative,decrease


In [ ]:
df_forSA['sentiment'].value_counts()

negative    1031
positive     769
Name: sentiment, dtype: int64

In [ ]:
#save the data we sentiment ans stock_price columns
df_forSA = pd.read_csv("/content/gdrive/MyDrive/3owais_Capital/Earning_call/Final_labled_data.csv")
pdf = pd.DataFrame(df_forSA)

def preprocess_text(text):
    text = re.sub(r'\[[^\]]+\]', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s%]', '', text)
    text = text.lower()
    return text

# Apply the preprocess_text function and create a new column
pdf['preprocessed_transcript'] = pdf['transcript'].apply(preprocess_text)
pdf =pdf.drop(columns=['Unnamed: 0'])

pdf.head(50000)
pdf.to_csv("/content/gdrive/MyDrive/3owais_Capital/Earning_call/FINAL_DATASET_PROCESSED_SUMMARIZED_NER_LABELED.csv")

In [ ]:
#READ the data to check
final = pd.read_csv("/content/gdrive/MyDrive/3owais_Capital/Earning_call/FINAL_DATASET_PROCESSED_SUMMARIZED_NER_LABELED.csv")
final['preprocessed_transcript'][1]

'thank you for standing by this is the conference operator welcome to the griffon corporations third quarter 2020 earnings conference call  i would now like to turn the conference over to brian harris chief financial officer please go aheadbrian g harris\xa0\xa0senior vice president and chief financial officerthank you good afternoon everyone with me on the call is ron kramer our chairman and chief executive officer our call is being recorded and will be available for playback the details of which are in our press release issued earlier todayas in the past our comments will include forwardlooking statements about the companys performance based on our views of griffons businesses and the environments in which they operate such statements are subject to inherent risks and uncertainties that can change as the world changes please see the cautionary statements in todays press release and in our various securities and exchange commission filingsfinally some of todays remarks will adjust for

# DONE ✅✅✅✅